<a href="https://colab.research.google.com/github/Yeyeong99/DevJS/blob/main/AI/RAG_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv

filename = '/content/drive/MyDrive/DevJS/데이터 구축_ 예영/2_selfintro_dataset.csv'
data = []

with open(filename, mode='r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    next(reader)  # 헤더 건너뛰기
    for row in reader:
        data.append(
            {
                "Highlight": row[0],
                "SelfIntroduction": row[1],
                "Feedback": row[2],
                "JDReflectRate": row[3],
                "JobCategory": row[4]
            }
        )

print(data)

[{'Highlight': 'Rust를 활용한 백엔드 개발 경험', 'SelfIntroduction': '저는 Rust 언어에 관심이 있어 백엔드 개발을 배우고 있습니다. Rust의 기능을 익히고 싶은 의지가 있습니다.', 'Feedback': "5점 만점으로 평가하였을 때 1점입니다.  실제 Rust를 활용한 백엔드 개발 경험에 대한 내용이 전혀 없어,  요청된 핵심 역량을 보여주지 못했습니다. 'Rust 언어에 관심이 있어 백엔드 개발을 배우고 있습니다.'와 같은 일반적인 표현보다는, 구체적인 경험과 그 경험을 통해 얻은 성과나 배움을 강조하시고, 실무에 어떻게 적용할 수 있을지 반영하면 좋을 것 같습니다.", 'JDReflectRate': '1', 'JobCategory': 'Backend'}, {'Highlight': 'Rust를 활용한 백엔드 개발 경험', 'SelfIntroduction': '개인 프로젝트에서 Rust를 사용하여 백엔드 개발을 경험했으며, 웹 백엔드 개발에 있어 데이터베이스 연동을 구현해보았습니다.', 'Feedback': '5점 만점으로 평가하였을 때 2점입니다. Rust를 사용한 백엔드 개발 경험을 언급하고 있지만, 구체적인 프로젝트나 기술 사용 사례가 부족합니다. 또한, 배운 내용을 어떻게 실무에 적용할 수 있을지에 관한 언급이 필요합니다.', 'JDReflectRate': '2', 'JobCategory': 'Backend'}, {'Highlight': 'Rust를 활용한 백엔드 개발 경험', 'SelfIntroduction': '학부 시절 Rust 프로그래밍 언어를 학습하고, 졸업 프로젝트로 Rust를 사용하여 웹 애플리케이션의 백엔드를 구현했습니다. 이 프로젝트를 통해 Rust의 안정성과 성능에 대한 기본적인 이해를 갖추게 되었습니다.', 'Feedback': '5점 만점으로 평가하였을 때 3점입니다. Rust를 사용한 백엔드 개발 경험을 갖추고 있지만, 구체적인 예시나 성과가 부족합니다. Rust를 사용하여 실제 업무에서

In [3]:
import json

def save_as_jsonl(data_list, output_path="rag_data.jsonl"):
    with open(output_path, "w", encoding="utf-8") as f:
        for i, item in enumerate(data_list):
            json_line = {
                "id": str(i + 1),
                "text": item["SelfIntroduction"],
                "metadata": {
                    "Highlight": item["Highlight"],
                    "Feedback": item["Feedback"],
                    "JDReflectRate": item["JDReflectRate"],
                    "JobCategory": item["JobCategory"]
                }
            }
            f.write(json.dumps(json_line, ensure_ascii=False) + "\n")

# 사용 예시
save_as_jsonl(data)

In [7]:
pip install jsonlines

In [8]:
import jsonlines

file_path = "/content/rag_data.jsonl"
data = []

with jsonlines.open(file_path) as reader:
    for obj in reader:
        data.append(obj)

# 확인
print(data[0])

{'id': '1', 'text': '저는 Rust 언어에 관심이 있어 백엔드 개발을 배우고 있습니다. Rust의 기능을 익히고 싶은 의지가 있습니다.', 'metadata': {'Highlight': 'Rust를 활용한 백엔드 개발 경험', 'Feedback': "5점 만점으로 평가하였을 때 1점입니다.  실제 Rust를 활용한 백엔드 개발 경험에 대한 내용이 전혀 없어,  요청된 핵심 역량을 보여주지 못했습니다. 'Rust 언어에 관심이 있어 백엔드 개발을 배우고 있습니다.'와 같은 일반적인 표현보다는, 구체적인 경험과 그 경험을 통해 얻은 성과나 배움을 강조하시고, 실무에 어떻게 적용할 수 있을지 반영하면 좋을 것 같습니다.", 'JDReflectRate': '1', 'JobCategory': 'Backend'}}


In [12]:
# Load model directly
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta")
model = AutoModel.from_pretrained("BM-K/KoSimCSE-roberta")

In [13]:
# 텍스트 예시
texts = [item["text"] for item in data]

In [14]:
# 임베딩 저장 리스트
embeddings = []

for text in texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        # 일반적으로 [CLS] 토큰의 출력 벡터를 사용 (첫 번째 토큰)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        embeddings.append(cls_embedding.squeeze().numpy())

In [18]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 23.9 MB/s eta 0:00:00


In [19]:
import faiss
import numpy as np
import pickle

# 벡터 DB 초기화
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)

# 벡터 삽입
index.add(np.array(embeddings))

# 메타데이터 따로 저장 (인덱스 순서에 맞게)
metadatas = [item["metadata"] for item in data]
with open("metadata.pkl", "wb") as f:
    pickle.dump(metadatas, f)

# 벡터 저장
faiss.write_index(index, "faiss.index")


In [85]:
# 메타데이터 검색
query = "Spring boot 기반 백엔드 개발 경험"

In [86]:
query_input = tokenizer([query], return_tensors="pt", truncation=True, padding=True)

In [87]:
with torch.no_grad():
    query_output = model(**query_input)
    # 일반적으로 [CLS] 토큰의 출력 벡터를 사용 (첫 번째 토큰)
    cls_embedding = query_output.last_hidden_state[:, 0, :]
    query_embedding = cls_embedding.squeeze().numpy()

In [88]:
D, I = index.search(np.array([query_embedding]), k=3)

In [89]:
for idx in I[0]:
    print(metadatas[idx])

{'Highlight': 'Rust를 활용한 백엔드 개발 경험', 'Feedback': '5점 만점으로 평가하였을 때 2점입니다. Rust를 사용한 백엔드 개발 경험을 언급하고 있지만, 구체적인 프로젝트나 기술 사용 사례가 부족합니다. 또한, 배운 내용을 어떻게 실무에 적용할 수 있을지에 관한 언급이 필요합니다.', 'JDReflectRate': '2', 'JobCategory': 'Backend'}
{'Highlight': 'Flutter 또는 Swift를 활용한 모바일 앱 개발 경험', 'Feedback': '5점 만점으로 평가하였을 때 3점입니다. 모바일 앱 개발 경험을 가지고 있지만, 구체적인 성과나 실제 업무에서의 적용 방향성이 부족합니다. 모바일 앱 개발 경험을 구체적으로 설명하고, 그 경험을 통해 얻은 성과나 배움을 강조하면 좋을 것 같습니다.', 'JDReflectRate': '3', 'JobCategory': 'Mobile'}
{'Highlight': 'Rust을 활용한 백엔드 개발 경험', 'Feedback': '5점 만점에서 2점입니다. Rust를 사용한 백엔드 개발 경험을 언급하고 있지만, 구체적인 프로젝트나 기술 사용 사례가 부족합니다. 실제 업무에서 Rust를 어떻게 적용할지에 대한 설명이 필요합니다.', 'JDReflectRate': '2', 'JobCategory': 'Backend'}


In [107]:
query = """
학부 시절 팀 프로젝트에서 CI/CD 파이프라인 구축과 무중단 배포 환경 설계를 맡으면서 AWS 기반 클라우드 인프라에 대한 관심과 실질적인 경험을 쌓게 되었습니다.
당시 우리는 Spring Boot로 개발한 웹 서비스를 안정적으로 운영하기 위해 AWS의 다양한 서비스를 활용했습니다. 먼저, EC2와 RDS를 이용해 기본적인 서버-DB 구조를 구성했고, 이후 CodePipeline과 CodeDeploy를 연동하여 GitHub에 Push되는 시점부터 자동으로 배포까지 이어지는 CI/CD 파이프라인을 구축했습니다. 이를 통해 수동 배포 과정에서 발생할 수 있는 오류를 줄이고, 배포 시간을 획기적으로 단축할 수 있었습니다.

또한, S3와 CloudFront를 이용해 정적 리소스를 분리하고 캐싱 처리를 하여 서비스의 응답 속도를 개선하였으며, IAM 정책 설정과 보안 그룹 구성을 통해 서비스 접근 권한을 세분화하고, 보안 수준을 강화했습니다.
CloudWatch를 통해 애플리케이션 로그와 서버의 상태를 실시간으로 모니터링하며, 문제가 발생했을 때 빠르게 대응하는 경험도 했습니다. 실제로 CPU 사용률이 급상승한 상황에서 경보 알림을 받고 EC2 인스턴스를 수동으로 스케일업했던 경험은 실무와 유사한 문제 해결 역량을 키우는 데 큰 도움이 되었습니다.

이러한 경험을 바탕으로, 실무에서도 클라우드 인프라를 설계하고 자동화하여 효율적으로 운영하는 역할에 빠르게 적응하고 기여할 수 있다고 확신합니다. 특히 DevOps 문화가 중요한 환경에서, 인프라와 애플리케이션의 경계를 이해하고 유기적으로 연결할 수 있는 역량을 갖추고 있다고 생각합니다.
"""
query_input = tokenizer([query], return_tensors="pt", truncation=True, padding=True)
with torch.no_grad():
    query_output = model(**query_input)
    # 일반적으로 [CLS] 토큰의 출력 벡터를 사용 (첫 번째 토큰)
    cls_embedding = query_output.last_hidden_state[:, 0, :]
    query_embedding = cls_embedding.squeeze().numpy()

In [95]:
D, I = index.search(np.array([query_embedding]), k=3)

In [100]:
retrieved_contexts = [
    {
        "text": texts[i],
        "metadata": metadatas[i]
    }
    for i in I[0]
]

In [135]:
retrieved_contexts

[{'text': '저는 클라우드 네이티브 애플리케이션 개발에 관심이 있습니다. 클라우드 컴퓨팅에 대한 일반적인 지식을 가지고 있습니다.',
  'metadata': {'Highlight': 'Cloud Native Application Development',
   'Feedback': "5점 만점으로 평가하였을 때 1점입니다. 클라우드 네이티브 애플리케이션 개발 경험에 대한 내용이 전혀 없어, 요청된 핵심 역량을 보여주지 못했습니다. '클라우드 네이티브 애플리케이션 개발에 관심이 있습니다.'와 같은 일반적인 표현보다는, 구체적인 경험과 그 경험을 통해 얻은 성과나 배움을 강조하시고, 실무에 어떻게 적용할 수 있을지 반영하면 좋을 것 같습니다.",
   'JDReflectRate': '1',
   'JobCategory': 'Development'}},
 {'text': '학부 시절 AWS 클라우드 플랫폼을 기반으로 실시간 데이터 처리를 위한 클라우드 서비스 개발 프로젝트에 참여했습니다. 특히, 대용량의 데이터를 효율적으로 처리하고 애플리케이션의 성능을 향상시키기 위해 AWS Lambda와 API Gateway를 활용하여 서버리스 아키텍처를 구현했습니다. 이와 같은 경험으로 실무에서 클라우드 서비스 개발에 기여하겠습니다. 예를 들어, 클라우드 서비스 개발시 다양한 조건을 기반으로 데이터를 실시간으로 분류 또는 가공해야 할 때, AWS Lambda를 활용한 이벤트 드리븐 아키텍처와 API Gateway를 활용한 마이크로 서비스 아키텍처 방식을 이용해 효율을 높이겠습니다. 또한, AWS 서비스를 활용한 클라우드 서비스 개발 방식으로 중복되는 로직을 간결하게 추상화함으로써, 코드의 유지보수성 향상에 기여하겠습니다.',
  'metadata': {'Highlight': 'AWS 기반 클라우드 서비스 개발 경험',
   'Feedback': '5점 만점으로 평가하였을 때 5점입니다. AWS 기반 클라우드 서비스 개발 경험을 구체적으로 설명하고, 관련 기술 (AWS L

In [151]:
# 예: 가장 유사한 자기소개서를 컨텍스트로 활용
contexts = "\n\n".join([
    f"- {c['text']}\n피드백: {c['metadata']['Feedback']}" for c in retrieved_contexts
])

prompt = f"""
당신은 IT 분야로 진로를 정한 취업준비생들의 자기소개서를 첨삭해주는 IT 전문 컨설턴트입니다.
다음은 특정 JD에 대한 자기소개서 예시입니다. JD 반영도를 높이고 싶은 사용자에게 반드시 한국어로만 조언을 작성해주세요.

JD 키워드: {retrieved_contexts[0]['metadata']["Highlight"]}
유사 자소서 및 피드백:
{contexts}

사용자의 자기소개서: {query}

답변 형식:
1. 자기소개서의 장점과 개선할 점을 구체적으로 설명해주세요.
2. JD 키워드를 반영하기 위한 구체적인 조언을 제공해주세요.
3. 최종적으로 수정된 자기소개서의 예시를 제공해주세요.
"""

In [152]:
print(prompt)


당신은 IT 분야로 진로를 정한 취업준비생들의 자기소개서를 첨삭해주는 IT 전문 컨설턴트입니다.
다음은 특정 JD에 대한 자기소개서 예시입니다. JD 반영도를 높이고 싶은 사용자에게 반드시 한국어로만 조언을 작성해주세요.

JD 키워드: Cloud Native Application Development
유사 자소서 및 피드백:
- 저는 클라우드 네이티브 애플리케이션 개발에 관심이 있습니다. 클라우드 컴퓨팅에 대한 일반적인 지식을 가지고 있습니다.
피드백: 5점 만점으로 평가하였을 때 1점입니다. 클라우드 네이티브 애플리케이션 개발 경험에 대한 내용이 전혀 없어, 요청된 핵심 역량을 보여주지 못했습니다. '클라우드 네이티브 애플리케이션 개발에 관심이 있습니다.'와 같은 일반적인 표현보다는, 구체적인 경험과 그 경험을 통해 얻은 성과나 배움을 강조하시고, 실무에 어떻게 적용할 수 있을지 반영하면 좋을 것 같습니다.

- 학부 시절 AWS 클라우드 플랫폼을 기반으로 실시간 데이터 처리를 위한 클라우드 서비스 개발 프로젝트에 참여했습니다. 특히, 대용량의 데이터를 효율적으로 처리하고 애플리케이션의 성능을 향상시키기 위해 AWS Lambda와 API Gateway를 활용하여 서버리스 아키텍처를 구현했습니다. 이와 같은 경험으로 실무에서 클라우드 서비스 개발에 기여하겠습니다. 예를 들어, 클라우드 서비스 개발시 다양한 조건을 기반으로 데이터를 실시간으로 분류 또는 가공해야 할 때, AWS Lambda를 활용한 이벤트 드리븐 아키텍처와 API Gateway를 활용한 마이크로 서비스 아키텍처 방식을 이용해 효율을 높이겠습니다. 또한, AWS 서비스를 활용한 클라우드 서비스 개발 방식으로 중복되는 로직을 간결하게 추상화함으로써, 코드의 유지보수성 향상에 기여하겠습니다.
피드백: 5점 만점으로 평가하였을 때 5점입니다. AWS 기반 클라우드 서비스 개발 경험을 구체적으로 설명하고, 관련 기술 (AWS Lambda, API Gateway, Serverless) 을 활용한 사례를

In [140]:
pip install groq

In [153]:
from groq import Groq

In [154]:
client = Groq(api_key='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')
MODEL_NAME = "gemma2-9b-it"
messages = [
    {"role": "system", "content": "당신은 IT 분야로 진로를 정한 취업준비생들의 자기소개서를 첨삭해주는 IT 전문 컨설턴트입니다. 반드시 한국어로만 답변하세요."},
    {"role": "user", "content": prompt}
]

In [155]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
)

print(response.choices[0].message.content)

## 자기소개서 첨삭 및 개선 조언

**1. 장점 및 개선할 점**

**장점:**

*  AWS 서비스를 활용한 실무 경험을 구체적으로 설명하고 있습니다. 
* CI/CD 파이프라인 구축, 무중단 배포 환경 설계, 서버-DB 구축, 정적 리소스 분리 및 캐싱, 보안 설정, 모니터링 등 다양한 클라우드 인프라 기술을 접목한 경험을 제시합니다.
* DevOps 문화 이해와 인프라와 애플리케이션 연관성을 강조하여 적극적인 역량을 보여줍니다. 

**개선할 점:**

*  `Cloud Native Application Development` 키워드를 JD에 강조하는 부분이 부족합니다. 단순히 AWS 기반 인프라 구축 경험보다는, **클라우드 네이티브한 시각 with AWS** 으로 애플리케이션을 구축 및 운영한 경험을 강조하는 것이 좋습니다.
*  `Scale up` 에 대한 구체적인 기술 설명은 개발 프로세스와 직접적으로 연결되어 있지 않습니다. 서버 규모를 확장하는 대신, **프로젝트를 통해 궁극적으로 서비스 확장성, 가용성, 회복력을 향상시키도록 노력**했다는 의지를 보여주는 것이 중요합니다.
*  제시된 기술들은 모두 중요하지만, **핵심 기술에 대한 구체적인 이야기** 를 통해 JD에 부합하는 강점을 명확히 드러내야 합니다.



**2. JD 키워드 반영을 위한 조언**

*  자기소개서에서 `Cloud Native Application Development` 는 **자원의 가용성과 무한 확장성을 바탕으로 개발되는 애플리케이션** 로 이해해야 합니다.  AWS 서비스를 활용하여 **정적 리소스 분리와 캐싱** 처리뿐만 아니라 **서버리스 아키텍처**, 
    **미래에 확장 및 확장성을 갖춘 애플리케이션 설계로 나아간 경험** 을  보여주는 것이 중요합니다.
*  AWS Lambda , API Gateway 와 같은 **서비스를 통해 고객 경험을 향상시키는 구체적인 사례** 를 언급하며,  **새로운 기술을 배우고 적용하는 적극성**을 보여주세요.



In [146]:
prompt = f"""
당신은 IT 분야로 진로를 정한 취업준비생들의 자기소개서를 첨삭해주는 IT 전문 컨설턴트입니다.
다음은 특정 JD에 대한 자기소개서 예시입니다. JD 반영도를 높이고 싶은 사용자에게 반드시 한국어로만 조언을 작성해주세요.

JD 키워드: AWS 클라우드 인프라 설계 및 운영 경험

사용자의 자기소개서: {query}

답변 형식:
1. 자기소개서의 장점과 개선할 점을 구체적으로 설명해주세요.
2. JD 키워드를 반영하기 위한 구체적인 조언을 제공해주세요.
3. 최종적으로 수정된 자기소개서의 예시를 제공해주세요.
"""

In [148]:
messages = [
    {"role": "system", "content": "당신은 IT 분야로 진로를 정한 취업준비생들의 자기소개서를 첨삭해주는 IT 전문 컨설턴트입니다. 반드시 한국어로만 답변하세요."},
    {"role": "user", "content": prompt}
]

In [149]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
)

print(response.choices[0].message.content)

## 자기소개서 첨삭

**1. 장점과 개선점**

**장점:**

* 자기소개서에 제시된 팀 프로젝트 경험을 통해 AWS 기반 클라우드 인프라 설계 및 운영 경험을 보여주고 있습니다. 
* 사용한 AWS 서비스 (EC2, RDS, CodePipeline, CodeDeploy, S3, CloudFront 등)를 구체적으로 언급하며, 각 서비스의 역할과 활용 방식을 설명하고 있습니다.
* 실제로 수동 배포 과정에서 발생할 수 있는 문제를 해결하고 배포 시간을 단축하는 등 구체적인 결과를 제시하여 경험을 뒷받침하고 있습니다.
* DevOps 문화와 인프라와 애플리케이션의 경계 이해에 대한 역량 언급은 긍정적입니다.

**개선점:**

*  "AWS 클라우드 인프라 설계 및 운영 경험"이라는  JD 키워드를 강조하는 부분이 부족합니다. 
*  AWS 교육 경험이나 개인 프로젝트 등 추가적인 경험이 불충분합니다.
*  주관적인 표현 ("실무와 유사한 문제 해결 역량을 키우는 데 큰 도움이 되었습니다")보다 구체적인 성과와 실질적 영향을 강조하는 것이 좋습니다.


**2. JD 키워드 반영을 위한 조언**

* **AWS 클라우드 인프라 설계 및 운영 경험 언급 강조:**
    * 직접적인 표현 ( “AWS 클라우드 인프라 설계 및 운영 경험”을 명확히 언급)
    * 클라우드 인프라 설계/운영 관련 기술을 사용한 프로젝트 예시 강조
* **구체적인 기술 스킬 세부 활용 언급:**
    * JSON, YAML, Ansible, Terraform, Docker 등 JD에 언급된 기술 관련 경험 포함

* **AWS 인증, 교육, 개인 프로젝트 등 추가 경험 제시:** 
    * AWS 관련 인증 취득 상태(AWS Certified Solutions Architect, Developer 등)
    * AWS 기반 개인 프로젝트 경험
    * 온라인 교육 플랫폼 (A Cloud Guru, Udemy 등) 제공 서버 관리 및 자동화 기술 학습 경험  

**3. 